<a href="https://colab.research.google.com/github/hanaluw/NLP-Topic-Modeling-for-Vietnames-Fintech-news/blob/main/Bertopic_with_ctfidf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install transformers
# !pip install sentence_transformers
# !pip install pandas openpyxl  # đọc excel
!pip install -U bertopic

# Model

In [ ]:
import pandas as pd
df = pd.read_excel('/content/trial2018only.xlsx')
df.head()

In [ ]:
df.info()

In [ ]:
import re
import pandas as pd

def clean1(text):
    if not isinstance(text, str) or text.strip() == "":
        return ""
    text = re.sub(r"[“”‘’…\"']", " ", text)  # Remove special quotes
    text = re.sub(r"\s+", " ", text).strip()  # Normalize whitespace
    return text

# Apply cleaning only to valid strings
df['title'] = df['title'].astype(str).apply(clean1)
df['content'] = df['content'].astype(str).apply(clean1)

# Remove rows with empty content or title after cleaning
df = df[(df['title'].str.strip() != "") & (df['content'].str.strip() != "")]

# Convert to list for later use
title = df["title"].tolist()
content = df["content"].tolist()

# Ensure date column is datetime
df["date"] = pd.to_datetime(df["date"], errors="coerce")  # coerce invalid formats to NaT
df = df[df["date"].notna()]  # Drop rows where date conversion failed
timestamps = df["date"].tolist()

In [ ]:
import pickle
import os

# Create folder in Drive
folder_path = "/content/drive/MyDrive/Fintech_BERTopic"
os.makedirs(folder_path, exist_ok=True)

# Save all data in one file
data = {
    "title": df["title"].tolist(),
    "content": df["content"].tolist(),
    "timestamps": df["date"].tolist()
}

with open(f"{folder_path}/text_data.pkl", "wb") as f:
    pickle.dump(data, f)

##load content back

In [ ]:
import pickle

with open("/content/drive/MyDrive/Fintech_BERTopic/text_data.pkl", "rb") as f:
    data = pickle.load(f)

title = data["title"]
content = data["content"]
timestamps = data["timestamps"]

In [ ]:
from transformers import AutoTokenizer

# Tải tokenizer của mô hình
tokenizer = AutoTokenizer.from_pretrained("dangvantuan/vietnamese-document-embedding", trust_remote_code=True)

# Tìm số token của mỗi văn bản
token_lengths = [len(tokenizer.encode(text, truncation=False)) for text in content]

# Tìm chiều dài lớn nhất và chỉ số của văn bản đó
max_length = max(token_lengths)
max_index = token_lengths.index(max_length)

print(f"Chiều dài lớn nhất (số token): {max_length}")
print(f"Văn bản dài nhất ở chỉ số: {max_index}")

# Kiểm tra có bị cắt không
if max_length > 8192:
    print("\n Văn bản này vượt quá giới hạn 8192 tokens ⇒ sẽ bị cắt khi embed.")
else:
    print("\n Văn bản nằm trong giới hạn ⇒ sẽ không bị cắt.")

#Pre-calculate Embeddings

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('dangvantuan/vietnamese-document-embedding', trust_remote_code=True, device='cuda')

In [ ]:
embeddings = model.encode(content, batch_size=16, show_progress_bar=True)

In [ ]:
import numpy as np

# After computing embeddings
np.save('fintechembeddings.npy', embeddings)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import os

folder_path = "/content/drive/MyDrive/BERTopc_Light"
os.makedirs(folder_path, exist_ok=True)

In [ ]:
np.save(f"{folder_path}/fintechembeddings.npy", embeddings)

##load embeddings back

In [ ]:
import numpy as np

# Load the embeddings
embeddings = np.load('/content/fintechembeddings.npy')

# Optional: check shape
print(embeddings.shape)

# Dimensionality reduction + Clustering

In [ ]:
from bertopic import BERTopic
from umap import UMAP
umap_model = UMAP(
    n_neighbors=30,
    n_components=5,
    min_dist=0.05,
    metric='cosine',
    random_state=42,
    low_memory=True,
    n_jobs=-1
)

In [ ]:
from hdbscan import HDBSCAN
hdbscan_model = HDBSCAN(
    min_cluster_size=15,            # Minimum number of docs in a topic
    metric='euclidean',             # Works well with UMAP (cosine already applied in UMAP)
    cluster_selection_method='eom', # Standard for well-separated clusters
    prediction_data=True,           # Needed if you plan to update or visualize
    core_dist_n_jobs=-1             # Use all CPU cores
)

In [ ]:
from bertopic import BERTopic

In [ ]:
!pip install underthesea

In [ ]:
from underthesea import word_tokenize

In [ ]:
vietnamese_stopwords = [
    "và", "của", "là", "có", "cho", "trong", "được", "với", "một", "những",
    "các", "đã", "đang", "đến", "này", "từ", "ra", "vào", "nếu", "cũng",
    "như", "làm", "khi", "thì", "vì", "tại", "vậy", "nhưng", "để", "cần",
    "qua", "nên", "sau", "đó", "vẫn", "nhiều", "năm", "đi", "đó", "ai",
    "bao", "bằng", "chỉ", "có thể", "giữa", "hay", "kẻ", "không", "lại",
    "lên", "lúc", "mà", "nào", "nữa", "phải", "qua", "ra", "rằng", "rất",
    "tất cả", "thế", "thấy", "theo", "thì", "trên", "trước", "tuy", "và",
    "vậy", "vì", "với", "đã", "đang", "đến", "điều", "đó", "được", "đây",
    "đó", "để", "ở", "ở đây", "ở đó", "ấy", "ấy là", "đừng", "không", "này",
    "này là", "đều", "như", "đến", "bởi", "đã", "làm", "ra", "với","người",
    "ông","bà","có","đây","bị","khi","là","của","tại","và","do","theo","với","hơn",
    "trong","về","một","những","ngoài_ra","cũng","đã","rằng","trên","đó",
    "không","chỉ","nhưng","như","các","sẽ","cùng","còn","giúp","được","nếu",
    "dù","mà","qua","bên_cạnh_đó","tuy_nhiên","song","bởi","như_vậy",
    "đồng_thời","vậy_nên","bởi_vậy","bởi_vì","thế_nên","thế_nhưng","đâu",
    "đâu_đó","tất_cả","điều_này","việc_này","này","ấy","nào","gì","vậy","thế",
    "rất","cả","mỗi","hết","bất_cứ","mọi","tuy","mỗi_khi","hễ","thật","quả_thật",
    "chính","gần","xa","hết_sức","cực_kỳ","vô_cùng","lắm","quá","bao_nhiêu",
    "nhiều","ít","nào_đó","mình","cho","việc","tin","mức","đầu","cuối","phải",
    "lên","lớn","số","ra","biết","gửi","đạt","cần","vụ","đi","tới","mang","rõ",
    "cách","phương","cuộc","cạnh","thành","đủ","gồm","tiếp_tục","sử_dụng","nhận",
    "lần","nhóm","lượng","trả","tuần","nói","vừa","%","+","-","*","/","=","<",">",
    "&","tôi",",",".","(",")",":",";","bạn"]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Faster tokenizer using underthesea
def tokenize_vi(text):
    return word_tokenize(text, format="text").split()

vectorizer_model = CountVectorizer(
    tokenizer=tokenize_vi,
    stop_words=vietnamese_stopwords,
    min_df=3, #minimum document frequency threshold
    ngram_range=(1, 2),
    max_features=30000
)

In [ ]:
from bertopic.vectorizers import ClassTfidfTransformer
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

#BERTopic training

In [ ]:
from bertopic import BERTopic

topic_model = BERTopic(

  # Pipeline models
  embedding_model=model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  vectorizer_model=vectorizer_model,
  ctfidf_model=ctfidf_model,

  # Hyperparameters
  top_n_words=50, #Show top 30 most relevant words .get_topic_info()
  verbose=True
)

topics, probs = topic_model.fit_transform(content, embeddings)

#LET'S GET STARTED

In [ ]:
# In ra 10 topic phổ biến nhất
topic_model.get_topic_info()

In [ ]:
rep_docs = topic_model.representative_docs_

# Create a map from doc text to title for quick lookup
doc_to_title = {doc_text: doc_title for doc_text, doc_title in zip(content, title)}

# Print representative docs + their titles by topic
for topic_id, docs in rep_docs.items():
    print(f"Topic {topic_id}:")
    for doc_text in docs:
        doc_title = doc_to_title.get(doc_text, "Title not found")
        print(f"  Title: {doc_title}")
    print("\n")

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

def get_ctfidf_similarity_scores(topic_model, docs, topics):
    # Bước 1: Lấy vectorizer và mô hình c-TF-IDF
    vectorizer = topic_model.vectorizer_model
    ctfidf_model = topic_model.ctfidf_model

    # Bước 2: Transform văn bản thành BoW rồi c-TF-IDF vector
    X_bow = vectorizer.transform(docs)
    X_ctfidf = ctfidf_model.transform(X_bow)

    # Bước 3: Lấy ma trận c-TF-IDF của các topic
    topic_ctfidf = topic_model.c_tf_idf_
    topic_info = topic_model.get_topic_info()
    topic_id_to_index = {tid: i for i, tid in enumerate(topic_info["Topic"].tolist())}

    # Bước 4: Tính similarity giữa mỗi văn bản với topic được gán
    similarity_scores = []
    for i in range(len(docs)):
        topic_id = topics[i]
        if topic_id == -1 or topic_id not in topic_id_to_index:
            similarity_scores.append(None)
        else:
            topic_index = topic_id_to_index[topic_id]
            doc_vec = X_ctfidf[i]
            topic_vec = topic_ctfidf[topic_index]
            sim = cosine_similarity(doc_vec, topic_vec)[0][0]
            similarity_scores.append(sim)

    return similarity_scores

In [ ]:
similarity_scores = get_ctfidf_similarity_scores(topic_model, content, topics)

df_similarity = pd.DataFrame({
    "Title": title,
    "Assigned Topic": topics,
    "Similarity to Topic (c-TF-IDF)": similarity_scores
})


In [ ]:
df_similarity

In [ ]:
df_similarity.to_excel("similarity_scores.xlsx", index=False, engine="openpyxl")

# Load for visual

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

folder_path = "/content/drive/MyDrive/BERTopic_Light"
print(os.listdir(folder_path))

In [ ]:
from bertopic import BERTopic

In [ ]:
loaded_model = BERTopic.load("/content/drive/MyDrive/BERTopic_Light")

In [ ]:
loaded_model.visualize_hierarchy()

In [ ]:
fig1 =loaded_model.visualize_hierarchy()
fig1.write_html("/content/hierarchy_final.html")
fig1